In [2]:
%matplotlib qt 
import matplotlib.pyplot as plt
import cv2
import numpy as np
import math

Part 2.3:

In [3]:
def imgprocess(img):
    img = img.astype(np.float32)
    if img.max() > 1.5: img /= 255.0
    return img

def samesizer(A, B):  # with interpolation
    if A.shape[:2] != B.shape[:2]:
        B = cv2.resize(B, (A.shape[1], A.shape[0]), interpolation=cv2.INTER_AREA)
    return A, B

def gaussianStack(img, levels=5, sigma=2.0):  # not downsampling likle in pyramid
    img = imgprocess(img)
    G = [img]
    for _ in range(1, levels): #apply gaussian filter at each level
        G.append(cv2.GaussianBlur(G[-1], (0,0), sigmaX=sigma, sigmaY=sigma,
                                  borderType=cv2.BORDER_REFLECT))
    return G

def laplacianStack(G):
    L = [G[i] - G[i+1] for i in range(len(G)-1)]
    L.append(G[-1].copy())
    return L

def buildLaplacian(L): 
    out = np.zeros_like(L[0], dtype=np.float32)
    for Li in L: out += Li
    return np.clip(out, 0.0, 1.0)

def prepMask(m, like_img):
    m = imgprocess(m)
    if m.ndim == 3 and m.shape[2] == 3:
        m = 0.2126*m[...,0] + 0.7152*m[...,1] + 0.0722*m[...,2]
    if like_img.ndim == 3:
        m = m[..., None]   # HxW -> HxWx1
    return np.clip(m, 0.0, 1.0)

def multiresBlend(A, B, blend_mask=None, levels=5, sigma_img=2.0, sigma_mask=8.0):
    A = imgprocess(A); B = imgprocess(B)
    A, B = samesizer(A, B)

    if blend_mask is None:
        H, W = A.shape[:2]
        blend_mask = np.zeros((H,W), np.float32); blend_mask[:, :W//2] = 1.0

    GA, GB = gaussianStack(A, levels, sigma_img), gaussianStack(B, levels, sigma_img)
    LA, LB = laplacianStack(GA), laplacianStack(GB)

    # base mask level
    M0 = prepMask(blend_mask, A) 
    GM = [M0]

    # smooth mask per level; IMPORTANT TO MAKE SURE Channel axis exists for color images otherwise wont stack properly
    for _ in range(1, levels):
        Mi = cv2.GaussianBlur(GM[-1], (0,0), sigmaX=sigma_mask, sigmaY=sigma_mask,
                              borderType=cv2.BORDER_REFLECT)
        if A.ndim == 3 and Mi.ndim == 2:   # OpenCV dropped the channel axis...pls do better
            Mi = Mi[..., None]             # make it HxWx1 so it broadcasts to HxWx3
        GM.append(Mi)

    Lblend = []
    for i in range(levels):
        Mi = GM[i]
        if LA[i].ndim == 3 and Mi.ndim == 2:
            Mi = Mi[..., None]
        Lblend.append(Mi * LA[i] + (1.0 - Mi) * LB[i])

    blended = buildLaplacian(Lblend)
    return blended, (GA, GB, LA, LB, GM, Lblend)

In [4]:
appl  = "media/lemon_cropped.jpg"
oran = "media/moon_cropped.jpg"

apple  = cv2.cvtColor(cv2.imread(appl,  cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
orange = cv2.cvtColor(cv2.imread(oran, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)

In [8]:
H, W = apple.shape[:2]
blend_mask = np.zeros((H, W), np.float32)
blend_mask[:, :W//2] = 1.0

#for 5 pyramid levels, changing sigma levels affects how smooth the transition is
blended, (GA, GB, LA, LB, GM, Lblend) = multiresBlend(apple, orange, blend_mask=blend_mask, levels=5, sigma_img=2, sigma_mask=20)

plt.figure(figsize=(12,10))
plt.subplot(1,3,1); plt.imshow(apple);   plt.title("Lemon");  plt.axis("off")
plt.subplot(1,3,2); plt.imshow(orange);  plt.title("Moon"); plt.axis("off")
plt.subplot(1,3,3); plt.imshow(blended); plt.title("Lemoon"); plt.axis("off")
plt.show()

output = (blended*255).astype(np.uint8)
cv2.imwrite("oraple.png", cv2.cvtColor(output, cv2.COLOR_RGB2BGR))

True

In [6]:
#for visualising and recreating fig3, rows of high low and medium frequencies for each of the 3 images
def channelalign(M, like_img):
    return M[..., None] if (like_img.ndim == 3 and M.ndim == 2) else M

def normalise(x):
    x = x.astype(np.float32)
    mn, mx = x.min(), x.max()
    return np.zeros_like(x) if mx <= mn else (x - mn) / (mx - mn)


def laplaceCenter(x):
    a = np.max(np.abs(x).astype(np.float32))
    y = 0.5 + (x / (2*a + 1e-8))
    return np.clip(y, 0, 1)

def show(ax, img, title=""):
    im = np.squeeze(img)
    if im.ndim == 2 or (im.ndim == 3 and im.shape[2] == 1):
        ax.imshow(im, cmap="gray", vmin=0, vmax=1)
    else:
        ax.imshow(im)
    ax.set_title(title); ax.axis("off")

def orapleLevel(LA, LB, GM, GA, GB, blended, picks=None):
    L = len(GA)
    if picks is None:
        picks = (0, L//2, L-1)   # high, mid, low frequencies
    rows = len(picks) + 1
    fig, axs = plt.subplots(rows, 3, figsize=(9, 3*rows))

    labels = ["high", "medium", "low"]
    for r, i in enumerate(picks):
        Mi = channelalign(GM[i], LA[i])
        left  = Mi * LA[i]
        right = (1.0 - Mi) * LB[i]
        both  = left + right
        tag = labels[r] if r < len(labels) else f"lvl {i}"
        show(axs[r, 0], laplaceCenter(left),f"({tag}) Lemon (L{i})")
        show(axs[r, 1], laplaceCenter(right),f"({tag}) Moon (L{i})")
        show(axs[r, 2], laplaceCenter(both),f"({tag}) Lemoon (L{i})")

    #cuz dont want black line seam
    Mcoarse = channelalign(GM[-1], GA[0])
    show(axs[-1, 0], normalise(Mcoarse * GA[0]), "Apple")
    show(axs[-1, 1], normalise((1.0-Mcoarse) * GB[0]), "Orange")
    show(axs[-1, 2], normalise(blended),"Full Oraple")
    plt.show()

In [7]:
orapleLevel(LA, LB, GM, GA, GB, blended, picks=(0,2,4))

Part 2.4:

In [33]:
banan = "media/banana.jpg"
dolph = "media/dolphinfixx.png"

banana  = cv2.cvtColor(cv2.imread(banan,  cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
dolphin = cv2.cvtColor(cv2.imread(dolph, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)

In [57]:
m = cv2.imread("media/mask2.jpg", cv2.IMREAD_GRAYSCALE)

In [65]:
_, m = cv2.threshold(m, 127, 255, cv2.THRESH_BINARY)

kernel = np.ones((3,3), np.uint8)
m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, kernel, iterations=1)
m = cv2.erode(m, kernel, iterations=2) 
m = cv2.GaussianBlur(m, (0,0), 1.5)

# 4) Convert to [0,1] and apply an S-curve so interior is very opaque
alpha = (m.astype(np.float32) / 255.0)
# S-curve: steeper transition, more opaque interior; tune k smaller -> sharper edge
k = 0.08
alpha = 1.0 / (1.0 + np.exp(-(alpha - 0.5)/k))
alpha = np.clip(alpha, 0.0, 1.0)

mask = alpha

In [66]:
blended, (GA, GB, LA, LB, GM, Lb) = multiresBlend(dolphin, banana,blend_mask=mask,levels=6,sigma_img=2.3,sigma_mask=3.5)

In [ ]:
plt.figure(figsize=(12,4))
plt.imshow(blended);       
plt.title("Dolphana"); plt.axis("off")
plt.show()

2025-09-26 17:18:29.845 python[11594:660513] +[CATransaction synchronize] called within transaction
